In [1]:
import java.io.{BufferedReader, InputStreamReader}

import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream
import org.apache.spark.input.PortableDataStream

In [1]:
import org.slf4j.LoggerFactory
val logger = LoggerFactory getLogger getClass.getName

logger = org.slf4j.impl.Log4jLoggerAdapter($iw)


org.slf4j.impl.Log4jLoggerAdapter($iw)

In [1]:
val fileNameFilter = Some(".*.json$")

fileNameFilter = Some(.*.json$)


Some(.*.json$)

In [1]:
val tarRDD = sc.binaryFiles("yelp-dataset-challenge/importer/src/test/resources/sample_yelp_dataset.tar.gz")

tarRDD = yelp-dataset-challenge/importer/src/test/resources/sample_yelp_dataset.tar.gz BinaryFileRDD[3] at binaryFiles at <console>:36


lastException: Throwable = null


yelp-dataset-challenge/importer/src/test/resources/sample_yelp_dataset.tar.gz BinaryFileRDD[3] at binaryFiles at <console>:36

In [1]:
tarRDD.flatMap {
    case (name: String, content: PortableDataStream) =>
        val tar = new TarArchiveInputStream(new GzipCompressorInputStream(content.open))
        Stream.continually(tar.getNextEntry)
            .takeWhile {
              case null => tar.close(); false
              case _ => true
            }
            .filter(!_.isDirectory)
            .filter(_.getSize > 0)
            .filter(entry => fileNameFilter.forall(entry.getName.matches(_)))
            .map(_.getName -> new BufferedReader(new InputStreamReader(tar)))
}
.collect()

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0.0 in stage 1.0 (TID 1) had a not serializable result: java.io.BufferedReader
Serialization stack:
	- object not serializable (class: java.io.BufferedReader, value: java.io.BufferedReader@4d9191af)
	- field (class: scala.Tuple2, name: _2, type: class java.lang.Object)
	- object (class scala.Tuple2, (business.json,java.io.BufferedReader@4d9191af))
	- element of array (index: 0)
	- array (class [Lscala.Tuple2;, size 6)
StackTrace: Serialization stack:
	- object not serializable (class: java.io.BufferedReader, value: java.io.BufferedReader@4d9191af)
	- field (class: scala.Tuple2, name: _2, type: class java.lang.Object)
	- object (class scala.Tuple2, (business.json,java.io.BufferedReader@4d9191af))
	- element of array (index: 0)
	- array (class [Lscala.Tuple2;, size 6)
  at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1887